File for testing purposes only.

In [7]:
import os
import subprocess
import sys
import zipfile as z
import requests
import json

try:
    from bioservices import BioModels
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "bioservices"])
    from bioservices import BioModels

modelsDir = '../models'
if not os.path.exists(modelsDir):
    os.makedirs(modelsDir)

s = BioModels()

WARNING [bioservices.BioModels:130]:  The URL (https://www.ebi.ac.uk/biomodels) provided cannot be reached.
INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


In [8]:
def create_query() -> str:
    query_parts_full = {
        'mode': '*:*',
        'species': 'TAXONOMY:9606',
        'curation_status': 'curationstatus:"Manually curated"',
        'formats': 'modelformat:"SBML"',
        'kw': 'submitter_keywords:"Immuno-oncology"'
    }

    query_parts = [value for value in query_parts_full.values() if value]
    query = " AND ".join(query_parts)

    query_parts_simple = {
        key: value.split(":")[1] if ":" in value else value
        for key, value in query_parts_full.items()
    }

    query_simple = " AND ".join(query_parts_simple.values())

    return query, query_simple

q, sq = create_query()
print(q)
print(sq)

*:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"
* AND 9606 AND "Manually curated" AND "SBML" AND "Immuno-oncology"


Final :

In [9]:
def get_ids() -> list:
    query, sq = create_query()

    print(f"Generated query: {query}")

    try:
        search_results = s.search(query)

        if not search_results or 'models' not in search_results or not search_results['models']:
            print("No models found.")
            return []

        model_ids = [model['id'] for model in search_results['models']]
        return model_ids

    except Exception as e:
        print(f"Error during search: {e}")
        return []

In [10]:
def download_biomodels(directory: str, model_ids: list):
    filenames = []
    num_per_download = 100

    if len(model_ids) > num_per_download:
        num_downloads = len(model_ids) // num_per_download
        remainder = len(model_ids) % num_per_download
        if remainder > 0:
            num_downloads += 1
    else:
        num_downloads = 1

    start = 0
    for download_number in range(1, num_downloads + 1):
        if download_number == num_downloads:
            end = len(model_ids)
        else:
            end = start + num_per_download

        fname = os.path.join(directory, f"Biomodels{start+1}-{end}.zip")
        filenames.append(fname)

        if os.path.isfile(fname):
            os.remove(fname)

        current_batch_ids = model_ids[start:end]

        s.search_download(current_batch_ids, output_filename=fname)
        print(f"Biomodels models from ID {start+1} to {end} saved to {fname}")

        start = end

    # Consolidate all downloaded ZIP files into one
    with z.ZipFile(filenames[0], 'a') as z1:
        for fname in filenames[1:]:
            zf = z.ZipFile(fname, 'r')
            for n in zf.namelist():
                z1.writestr(n, zf.open(n).read())

    # Rename the first zip to "biomodels.zip"
    biomodels_zip = os.path.join(directory, "biomodels.zip")
    if not os.path.isfile(biomodels_zip):
        os.rename(filenames[0], biomodels_zip)

    # Clean up the other zip files
    for i in range(1, len(filenames)):
        try:
            os.remove(filenames[i])
        except Exception:
            continue

    return filenames

Using the code to answer :

In [12]:
ACTIVATE_DOWNLOAD = True
query, sq = create_query()
model_ids = get_ids()
print(f"Model IDs: {model_ids}")
print(f"Found {len(model_ids)} models matching the query.")
if ACTIVATE_DOWNLOAD and model_ids:
    download_biomodels(modelsDir, model_ids)

Generated query: *:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"
Model IDs: ['BIOMD0000000919', 'BIOMD0000000921', 'BIOMD0000001011', 'BIOMD0000001012', 'BIOMD0000001013', 'BIOMD0000001014', 'BIOMD0000001022', 'BIOMD0000001023', 'BIOMD0000001024', 'BIOMD0000001025']
Found 10 models matching the query.


INFO    [bioservices.BioModels:240]:  ../models/Biomodels1-10.zip


Biomodels models from ID 1 to 10 saved to ../models/Biomodels1-10.zip
